On adapte l'article d'Acemoglu-Akcigit-Kerr à notre projet. Ainsi, nous modélisons la convergence technologique comme un processus cumulatif structuré par un réseau historique de proximités technologiques, mesuré par les cooccurrences passées de codes CPC.

On peut s’appuyer sur cette méthodologie des réseaux d’innovation en remplaçant les flux de citations par des cooccurrences de codes CPC sur un même brevet et qui mesurent une interaction technologique. Dans le modèle de l’article, le réseau passé structure l’innovation future et la prédit ; tandis que dans notre modèle, le réseau passé indiquera ce qui est banal, original, ou émergent.   L’innovation observée concernera pour nous la nouveauté combinatoire. Il nous sera ainsi possible de reprendre leur modèle économétrique en calculant les convergences prédites par le réseau : si des domaines voisins convergent fortement sur la période étudiée, alors ils sont davantage exposés à une convergence dans le futur. Cela nous amène à aborder la convergence technologique comme une dynamique cumulative expliquée par l’espace technologique hérité des années précédentes. 

In [2]:
import pandas as pd
import itertools
import numpy as np


In [ ]:
# à mettre dans le terminal : gdown https://drive.google.com/uc?id=1Zi2RFRuZZf_OWDaocgcxhFcOsjqWrxJv

In [3]:

base_brevets = pd.read_parquet("patents_merged.parquet")

In [4]:
base_brevets.head()

,doc_number,doc7,ep_doc,publication_date,cpc4_list,person_id_list,app_name_list,address_list,city_list,postal_code_list,reg_code_list,ctry_code_list,reg_share_list,app_share_list,num_applicants
0,1911230,1911230,EP-1911230,20110105,"[H04L, H04B]",[8964],[QUALCOMM Incorporated],"[5775 Morehouse Drive,San Diego, CA 92121]",[None],[92121],[US06073],[US],[1.0],[1.0],1
1,1852468,1852468,EP-1852468,20090805,"[C08G, H01L, C08K, C08L, C09D]",[10990],"[Shin-Etsu Chemical Co., Ltd.]","[6-1, Ohtemachi 2-chome, Chiyoda-ku,Tokyo]",[Tokyo],[None],[JPD13],[JP],[1.0],[1.0],1
2,1932561,1932561,EP-1932561,20100217,"[A61B, A61N]",[46971525],[Commissariat à l'Energie Atomique],"[25, rue Leblanc Immeuble 'Le Ponant D',75015 ...",[Paris],[75015],[FR101],[FR],[1.0],[1.0],1
3,1991453,1991453,EP-1991453,20170906,"[Y02T, B60W, B60L, B60K]",[450],[ROBERT BOSCH GMBH],"[Postfach 30 02 20,70442 Stuttgart]",[Stuttgart],[70442],[DE111],[DE],[1.0],[1.0],1
4,1977104,1977104,EP-1977104,20091021,[F02M],[25653],"[KIMBERLY-CLARK WORLDWIDE, INC.]","[401 North Lake Street,Neenah, WI 54956]",[None],[54956],[US55139],[US],[1.0],[1.0],1


In [5]:
len(base_brevets)

2382887

In [6]:
base_multi["year"] = base_multi["publication_date"].astype(str).str[:4].astype(int)

In [8]:
def extract_classe(c):
    return c[:3]

base_multi["liste_classes"] = base_multi["cpc4_list"].apply(
    lambda arr: list({extract_classe(c) for c in arr})
)

base_multi.head()

,doc7,doc_number,ep_doc,publication_date,cpc4_list,person_id_list,app_name_list,address_list,city_list,postal_code_list,reg_code_list,ctry_code_list,reg_share_list,app_share_list,num_applicants,year,liste_classes
0,0719740,719740,EP-0719740,19990519,"[C03C, Y10S]","[1357945, 80365, 1095194, 101435, 51193962, 17...","[GEA Food Solutions Germany GmbH, Yale Univers...","[Im Ruttert,35216 Biedenkopf-Wallau, Two Whitn...","[Biedenkopf, None, Strand, Oberdorf, Leverkuse...","[35216, 06511, WC2R2LS, 4436, 51373, 19101, V6...","[DE724, US09009, UKJ13, CH032, DEA24, US42101,...","[DE, US, GB, CH, DE, US, CA, DE, NL, AT, US, J...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[1.0, 0.5, 0.5, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",15,1999,"[Y10, C03]"
1,0719776,719776,EP-0719776,19981104,"[A61K, C07D, A61P]","[4529, 3542266, 48039808, 3772, 464889, 24625,...",[Shell Internationale Research Maatschappij B....,"[Carel van Bylandtlaan 30,2596 HR Den Haag, We...","[Den Haag, Oberndorf, Paris, Beerse, ISTANBUL,...","[2596, 78727, 75008, 2340, 34950, None, 20100,...","[NL332, DE135, FR101, BE213, TR100, LU000, FI1...","[NL, DE, FR, BE, TR, LU, FI, US, US]","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]",9,1998,"[A61, C07]"
2,0846888,846888,EP-0846888,20030813,"[Y10S, F16H]","[48137948, 369846, 4387578, 4285261, 49683332,...","[Gardner Denver Inc., Osaka Prefecture Univers...","[222 East Erie Street,Milwaukee, WI 53202, 1-1...","[None, osaka, osaka, Aichi, None, Gyeonggi do,...","[53202, 599, 541, 471, 80528, None, 53511, 219...","[US55079, JPG27, JPG27, JPF23, US08069, KR013,...","[US, JP, JP, JP, US, KR, US, ZA, HK]","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]","[1.0, 0.5, 0.5, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]",9,2003,"[Y10, F16]"
3,0846899,846899,EP-0846899,20030827,"[Y10T, B60K, F16H]","[49749440, 5993, 1235]","[Samsung Electronics Co., Ltd., Olympus Corpor...","[129, Samsung-ro,Yeongtong-gu,Suwon-si, Gyeong...","[Gyeonggi do, Tokyo, Hamburg]","[443742, 151, 21129]","[KR013, JPD13, DE600]","[KR, JP, DE]","[1.0, 1.0, 1.0]","[1.0, 1.0, 1.0]",3,2003,"[Y10, F16, B60]"
4,0846906,846906,EP-0846906,20010725,"[F16B, Y10T, F16L]","[1717384, 48047455, 48082922, 49332326, 513454...","[Aktiebolaget SKF, Lund, Thore, Rydèn, Karin, ...","[41 550 Göteborg, Lundgatan 14,416 61 Göteborg...","[GoTEBORG, Goteborg, Goteborg, SEATTLE, Gyeong...","[41550, 41661, 41658, 1226, None, 530, 40503, ...","[SE232, SE232, SE232, US53033, KR013, JPG27, S...","[SE, SE, SE, US, KR, JP, SE, FR, JP]","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]","[0.333333333333333, 0.333333333333333, 0.33333...",9,2001,"[Y10, F16]"


On construit la matrice de coocurrence historique (2000-2010). Elle représente le réseau technologique hérité

In [ ]:
# Période de référence
df_ref = base_multi[(base_multi["year"] >= 2000) & (base_multi["year"] <= 2010)].copy()




In [11]:
# on compte les paires CPC
from collections import Counter

pair_counter = Counter()

for cpcs in df_ref["liste_classes"]:
    for i, j in itertools.combinations(sorted(set(cpcs)), 2):
        pair_counter[(i, j)] += 1
        pair_counter[(j, i)] += 1  # matrice orientée


In [13]:
# on transforme la matrice en dataframe
cooc_df = pd.DataFrame(
    [(i, j, c) for (i, j), c in pair_counter.items()],
    columns=["cpc_i", "cpc_j", "count"]
)
cooc_df.head()

,cpc_i,cpc_j,count
0,F16,Y10,2180
1,Y10,F16,2180
2,B60,F16,990
3,F16,B60,990
4,B60,Y10,1270


!! il faut retirer les doublons pour chaque paire...

On normalise comme dans la matrice M d'Acemoglu Mi->j = Cij/somme sur k des Cik

In [16]:
# Somme par CPC source
row_sums = cooc_df.groupby("cpc_i")["count"].sum().reset_index()
row_sums.columns = ["cpc_i", "row_sum"]

cooc_df = cooc_df.merge(row_sums, on="cpc_i")
cooc_df["M_ij"] = cooc_df["count"] / cooc_df["row_sum"]


In [17]:
cooc_df.head()

,cpc_i,cpc_j,count,row_sum_x,M_ij,row_sum_y,row_sum
0,F16,Y10,2180,8149,0.267517,8149,8149
1,Y10,F16,2180,36314,0.060032,36314,36314
2,B60,F16,990,6919,0.143084,6919,6919
3,F16,B60,990,8149,0.121487,8149,8149
4,B60,Y10,1270,6919,0.183553,6919,6919


On construit un score d'originalité d'une combinaison CPC

In [18]:
EPS = 1e-10
cooc_df["originality_pair"] = -np.log(cooc_df["M_ij"] + EPS)


In [19]:
orig_dict = {
    (row.cpc_i, row.cpc_j): row.originality_pair
    for _, row in cooc_df.iterrows()
}


On crée une fonction qui donne l'originalité d'un brevet

In [20]:
def originality_of_patent(cpcs, orig_dict):
    pairs = list(itertools.combinations(sorted(set(cpcs)), 2))
    scores = []
    for i, j in pairs:
        if (i, j) in orig_dict:
            scores.append(orig_dict[(i, j)])
        else:
            # combinaison jamais observée historiquement
            scores.append(-np.log(EPS))
    return np.mean(scores) if scores else np.nan


In [23]:
df_post = [base_multi["year"] > 2010].copy()

df_post["originality"] = df_post["liste_classes"].apply(
    lambda x: originality_of_patent(x, orig_dict)
)


TypeError: list indices must be integers or slices, not str